# Data Collection Sample

This sample demonstrates how users can extract data from the simulation. The data collection interface was inspired by [Mesa's data collection API](https://mesa.readthedocs.io/en/latest/apis/datacollection.html). Data collection is a very important feature of agent-based simulations. Usually, we want to measure some internal dynamics of the simulation. Neighborly's DataCollector resource helps users aggregate all their data tables into a single location.

This example tracks the wealth (amount of money) that characters have and prints the contents of the data table when the simulation concludes. Data tables are exported as Pandas DataFrames to help Neighborly integrate with existing Python data science workflows.

Here we collect data within a custom simulation system. Neighborly has a `DataCollectionSystemGroup`, which encapsulates and runs all the systems responsible for collecting data.

## Import dependencies

In [1]:
from dataclasses import dataclass
from typing import Any, Dict

from tqdm.notebook import trange

from neighborly import (
    Component,
    Neighborly,
    NeighborlyConfig,
    SimDateTime,
    SystemBase,
    World,
)
from neighborly.data_collection import DataCollector
from neighborly.decorators import component, system
from neighborly.systems import DataCollectionSystemGroup, UpdateSystemGroup

## Create our simulation instance

In [2]:
sim = Neighborly(NeighborlyConfig(seed=101))

## Create custom components and systems

We use the `@component` and `@system` decorators to easily add these class definitions to the simulation. Alternatively, we would use `sim.world.gameobject_manager.register_component(...)` and `sim.world.system_manager.add_system(...)`.

In [3]:
@component(sim.world)
@dataclass
class Actor(Component):
    name: str

    def to_dict(self) -> Dict[str, Any]:
        return {"name": self.name}


@component(sim.world)
@dataclass
class Money(Component):
    amount: int

    def to_dict(self) -> Dict[str, Any]:
        return {"amount": self.amount}


@component(sim.world)
@dataclass
class Job(Component):
    title: str
    salary: int

    def to_dict(self) -> Dict[str, Any]:
        return {"title": self.title, "salary": self.salary}


@system(sim.world, system_group=UpdateSystemGroup)
class SalarySystem(SystemBase):
    def on_update(self, world: World) -> None:
        for _, (job, money) in world.get_components((Job, Money)):
            money.amount += job.salary


@system(sim.world, system_group=DataCollectionSystemGroup)
class WealthReporter(SystemBase):
    def on_create(self, world: World) -> None:
        world.resource_manager.get_resource(DataCollector).create_new_table(
            "wealth", ("uid", "name", "year", "money")
        )

    def on_update(self, world: World) -> None:
        year = world.resource_manager.get_resource(SimDateTime).year
        data_collector = world.resource_manager.get_resource(DataCollector)
        for guid, (actor, money) in world.get_components((Actor, Money)):
            data_collector.add_table_row(
                "wealth",
                {
                    "uid": guid,
                    "name": actor.name,
                    "year": year,
                    "money": money.amount,
                },
            )

## Spawn characters

We spawn two characters into the simulation and give them `Money` and `Job` components so that they qualify for the `SalarySystem`.

In [4]:
alice = sim.world.gameobject_manager.spawn_gameobject(
    [Actor("Alice"), Money(0), Job("WacArnold's", 20_000)]
)

kieth = sim.world.gameobject_manager.spawn_gameobject(
    [Actor("Kieth"), Money(0), Job("McDonald's", 32_500)]
)

## Run the simulation

In [5]:
YEARS = 30

for _ in trange(YEARS):
    sim.step()

  0%|          | 0/30 [00:00<?, ?it/s]

## Display the wealth data

In [6]:
sim.world.resource_manager.get_resource(DataCollector).get_table_dataframe("wealth")

,uid,name,year,money
0,1,Alice,1,0
1,2,Kieth,1,0
2,1,Alice,2,20000
3,2,Kieth,2,32500
4,1,Alice,3,40000
5,2,Kieth,3,65000
6,1,Alice,4,60000
7,2,Kieth,4,97500
8,1,Alice,5,80000
9,2,Kieth,5,130000
